In [1]:
import findspark
findspark.init('/home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark')

In [2]:
from astropy.io import fits

In [3]:
#import astroprov
import numpy as np
import os

from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column

import collections
import subprocess

import time
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool

from functools import partial

from astropy.io import ascii
#from sqlalchemy import create_engine
import sqlite3
import mysql.connector
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf
import datetime
from sqlalchemy import create_engine
import sys


In [4]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/mj1e16/miniconda2/lib/python2.7/site-packages/pyspark/mysql-connector-java_8.0.16-1ubuntu16.04_all/usr/share/java/mysql-connector-java-8.0.16.jar  pyspark-shell'
config = SparkConf().setAll([('spark.executor.cores', '6'),('spark.cores.max', '6'),('spark.driver.memory','1g'),('spark.executor.memory', '500m'),("spark.sql.execution.arrow.enabled", "true")]) # ("spark.sql.execution.arrow.enabled", "true")


In [5]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)

In [6]:

mags = [7,6,5,4,3,2,1]
for mag in mags:
    dataframe_mysql = sqlContext.read.format("jdbc").option("url", "jdbc:mysql://localhost/Kepler").option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist_{}".format(mag)).option("user", "mj1e16").option("password", "[sqlT1G3R]").load()
    dataframe_mysql.registerTempTable('starlist_{}'.format(mag))

In [79]:
def innerJoin(tableName,provDir,totGross,totGrossOriginal,variableList,mag,diffSize=1,xlength=1015.,ylength=1085.):
    nsegs = 4
    #mag = variableList[6]
    temptableName = tableName[0:5]+tableName[-1]
    OGtemptableName = 'original'
    OGdf = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(OGtemptableName,abs(mag)))
    OGdf.registerTempTable(OGtemptableName+'match')
    sqlContext.cacheTable(OGtemptableName+'match')
    OGtotNumber = OGdf.count()
    
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(temptableName,abs(mag)))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    #print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    score = 0
    #totSeg = []
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+15
        xmax = int(xmin+xsegment)+15-10
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+15
            ymax = int(ymin+ysegment)+15-10
            
            OGdf = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(OGtemptableName+'match',xmin,xmax,ymin,ymax))
            OGnumber = OGdf.count()
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            number = df.count()
            simulated = number-OGnumber
            score += abs(simulated-62)
            variableList.append(simulated)
            
#             dftot = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName,xmin,xmax,ymin,ymax))
#             number = df.count()
#             #variableList.append(number)
#             totSeg.append(number)

            # need to create some place to write to

    
    
#    variableList.extend(totSeg)
    #astroprov.provcall([tableName,'starlist_{}'.format(abs(mag))],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(OGtemptableName+'match')
    sqlContext.dropTempTable(OGtemptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    sqlContext.uncacheTable(OGtemptableName)
    sqlContext.dropTempTable(OGtemptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableList.append(OGtotNumber)
    variableList.append(totGrossOriginal) 
    variableList.append(score)
    variableTuple = tuple(variableList)
    #print(variableTuple)
    return variableTuple


def makeConfig(valList,provDir,tableName='table',defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',attributeList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    catName = 'test1.cat'
    confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

def findObjects(confName,tableName,catName,provDir,defaultDir='/home/mj1e16/sextractor/sextractor-master/config/',imagename='/home/mj1e16/iraf/editedImage5000.fits',original='no'):
    
    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    if original == 'yes':
        temptableName = 'original'
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    sqlContext.cacheTable(temptableName)

    return totGross

def alltogethernow(valList,tableName,IMAGE,provDir,variableList,originalImage,mag):
    Names = makeConfig(valList,provDir,tableName=tableName)
    totGrossOriginal = findObjects(Names[0],Names[1],Names[2],provDir,imagename=originalImage,original='yes')
    totGross = findObjects(Names[0],Names[1],Names[2],provDir,imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],provDir,totGross,totGrossOriginal,variableList,mag)
    return finalTuple

In [16]:
def hotpantsQuality(image):
    hdu = fits.open(image)
    imData = hdu[0].data
    imData = abs(imData)
    totVal = sum(imData)
    totVal = sum(totVal)
    return totVal

In [75]:
valList = [np.linspace(1,10,10),np.linspace(1,9,9),['gauss_1.5_3x3.conv','gauss_2.0_3x3.conv','gauss_2.0_5x5.conv',
                                                    'gauss_2.5_5x5.conv','gauss_3.0_5x5.conv','gauss_3.0_7x7.conv',
                                                    'gauss_4.0_7x7.conv','gauss_5.0_9x9.conv','mexhat_1.5_5x5.conv',
                                                    'mexhat_2.0_7x7.conv','mexhat_2.5_7x7.conv','mexhat_3.0_9x9.conv',
                                                    'mexhat_4.0_9x9.conv','mexhat_5.0_11x11.conv','tophat_1.5_3x3.conv',
                                                    'tophat_2.0_3x3.conv','tophat_2.5_3x3.conv','tophat_3.0_3x3.conv',
                                                    'tophat_4.0_5x5.conv','tophat_5.0_5x5.conv']]

In [80]:
def evaluateImage(valList,simImage,OGImage,provDir,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME']):
    
    finalTableTuples = []
    t0 = time.time()

    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                name = smallName+'_'+str(x0)+'_'+str(x1)+'_'
                for x2 in range(len(valList[2])):
                    variableList = [float(valList[0][x0]),float(valList[1][x1]),valList[2][x2],ccd,median,minmag,norm]
                    fullname = name+str(x2)
                    fullValList = [valList[0][x0],valList[1][x1],valList[2][x2]]
                    print(fullname)
                    try:
                        finalTableTuples.append(alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,provDir=provDir,variableList=variableList,originalImage=OGImage,mag=minmag))
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    print(finalTableTuples)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),totNum=int(x[23]),
                                   totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='allSpaceResultsScore_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()

    print(time.time()-t0)


In [11]:
#sqlContext.cacheTable('starlist')

In [40]:
#sqlContext.sql('DROP TABLE IF EXISTS finalResults_44_1')
#valList = [[1.],[1.],[8.],[3.]]
#valList = [np.linspace(1,2,2),np.linspace(1,2,2),[8.],[3.]]

In [83]:
# ccd = [44,63,79]
# median = [0,1,2]
# mags = np.linspace(-7,-1,7)

#ccd = [44] #,63,79]
ccd = [63]
median = [1]
mags = [-7,-6,-5,-4,-3,-2,-1]


In [84]:

norms = []
for inc,c in enumerate(ccd):
    totVals = []
    for x in range(3):
        totVals.append(hotpantsQuality('/home/mj1e16/Simages/diff_{}_{}.fits'.format(c,x)))
    norms.append([float(y/totVals[1]) for y in totVals])
print(norms)

[[19.092633575825804, 1.0, 1.9041523992142741]]


In [85]:
#valList = [np.linspace(1,10,2),np.linspace(1,9,2),['gauss_1.5_3x3.conv']]

In [86]:
### Make function to replace starlist
for inc,c in enumerate(ccd):
    for inmed,med in enumerate(median):
        norm = norms[0][1]
        #subprocess.call(['mkdir','/home/mj1e16/keplerPhotometry/provDump/{}/{}'.format(c,med)])
        for mag in mags:
            sqlContext.cacheTable('starlist_{}'.format(abs(mag)))
            #subprocess.call(['mkdir','/data/mj1e16/provDump/{}/{}/{}'.format(c,med,mag)])
            ogim = '/home/mj1e16/Simages/diff_{}_{}.fits'.format(c,med)
            imName = '/home/mj1e16/Simages/diff_{}_{}{}_alt.fits'.format(c,med,mag)
            evaluateImage(valList,imName,ogim,'/data/mj1e16/provDump/{}/{}/{}/'.format(c,med,mag),'ccd_{}'.format(c),'median_{}'.format(med),norm,mag,smallName='allSpace_{}_{}_{}'.format(c,med,mag))
            sqlContext.uncacheTable('starlist_{}'.format(abs(mag)))

allSpace_63_1_-7_0_0_0
allSpace_63_1_-7_0_0_1
allSpace_63_1_-7_0_0_2
allSpace_63_1_-7_0_0_3
allSpace_63_1_-7_0_0_4
allSpace_63_1_-7_0_0_5
allSpace_63_1_-7_0_0_6
allSpace_63_1_-7_0_0_7
allSpace_63_1_-7_0_0_8
allSpace_63_1_-7_0_0_9
allSpace_63_1_-7_0_0_10
allSpace_63_1_-7_0_0_11
allSpace_63_1_-7_0_0_12
allSpace_63_1_-7_0_0_13
allSpace_63_1_-7_0_0_14
allSpace_63_1_-7_0_0_15
allSpace_63_1_-7_0_0_16
allSpace_63_1_-7_0_0_17
allSpace_63_1_-7_0_0_18
allSpace_63_1_-7_0_0_19
allSpace_63_1_-7_0_1_0
allSpace_63_1_-7_0_1_1
allSpace_63_1_-7_0_1_2
allSpace_63_1_-7_0_1_3
allSpace_63_1_-7_0_1_4
allSpace_63_1_-7_0_1_5
allSpace_63_1_-7_0_1_6
allSpace_63_1_-7_0_1_7
allSpace_63_1_-7_0_1_8
allSpace_63_1_-7_0_1_9
allSpace_63_1_-7_0_1_10
allSpace_63_1_-7_0_1_11
allSpace_63_1_-7_0_1_12
allSpace_63_1_-7_0_1_13
allSpace_63_1_-7_0_1_14
allSpace_63_1_-7_0_1_15
allSpace_63_1_-7_0_1_16
allSpace_63_1_-7_0_1_17
allSpace_63_1_-7_0_1_18
allSpace_63_1_-7_0_1_19
allSpace_63_1_-7_0_2_0
allSpace_63_1_-7_0_2_1
allSpace_63_1_

allSpace_63_1_-7_1_8_9
allSpace_63_1_-7_1_8_10
allSpace_63_1_-7_1_8_11
allSpace_63_1_-7_1_8_12
allSpace_63_1_-7_1_8_13
allSpace_63_1_-7_1_8_14
allSpace_63_1_-7_1_8_15
allSpace_63_1_-7_1_8_16
allSpace_63_1_-7_1_8_17
allSpace_63_1_-7_1_8_18
allSpace_63_1_-7_1_8_19
allSpace_63_1_-7_2_0_0
allSpace_63_1_-7_2_0_1
allSpace_63_1_-7_2_0_2
allSpace_63_1_-7_2_0_3
allSpace_63_1_-7_2_0_4
allSpace_63_1_-7_2_0_5
allSpace_63_1_-7_2_0_6
allSpace_63_1_-7_2_0_7
allSpace_63_1_-7_2_0_8
allSpace_63_1_-7_2_0_9
allSpace_63_1_-7_2_0_10
allSpace_63_1_-7_2_0_11
allSpace_63_1_-7_2_0_12
allSpace_63_1_-7_2_0_13
allSpace_63_1_-7_2_0_14
allSpace_63_1_-7_2_0_15
allSpace_63_1_-7_2_0_16
allSpace_63_1_-7_2_0_17
allSpace_63_1_-7_2_0_18
allSpace_63_1_-7_2_0_19
allSpace_63_1_-7_2_1_0
allSpace_63_1_-7_2_1_1
allSpace_63_1_-7_2_1_2
allSpace_63_1_-7_2_1_3
allSpace_63_1_-7_2_1_4
allSpace_63_1_-7_2_1_5
allSpace_63_1_-7_2_1_6
allSpace_63_1_-7_2_1_7
allSpace_63_1_-7_2_1_8
allSpace_63_1_-7_2_1_9
allSpace_63_1_-7_2_1_10
allSpace_63_1

allSpace_63_1_-7_3_7_18
allSpace_63_1_-7_3_7_19
allSpace_63_1_-7_3_8_0
allSpace_63_1_-7_3_8_1
allSpace_63_1_-7_3_8_2
allSpace_63_1_-7_3_8_3
allSpace_63_1_-7_3_8_4
allSpace_63_1_-7_3_8_5
allSpace_63_1_-7_3_8_6
allSpace_63_1_-7_3_8_7
allSpace_63_1_-7_3_8_8
allSpace_63_1_-7_3_8_9
allSpace_63_1_-7_3_8_10
allSpace_63_1_-7_3_8_11
allSpace_63_1_-7_3_8_12
allSpace_63_1_-7_3_8_13
allSpace_63_1_-7_3_8_14
allSpace_63_1_-7_3_8_15
allSpace_63_1_-7_3_8_16
allSpace_63_1_-7_3_8_17
allSpace_63_1_-7_3_8_18
allSpace_63_1_-7_3_8_19
allSpace_63_1_-7_4_0_0
allSpace_63_1_-7_4_0_1
allSpace_63_1_-7_4_0_2
allSpace_63_1_-7_4_0_3
allSpace_63_1_-7_4_0_4
allSpace_63_1_-7_4_0_5
allSpace_63_1_-7_4_0_6
allSpace_63_1_-7_4_0_7
allSpace_63_1_-7_4_0_8
allSpace_63_1_-7_4_0_9
allSpace_63_1_-7_4_0_10
allSpace_63_1_-7_4_0_11
allSpace_63_1_-7_4_0_12
allSpace_63_1_-7_4_0_13
allSpace_63_1_-7_4_0_14
allSpace_63_1_-7_4_0_15
allSpace_63_1_-7_4_0_16
allSpace_63_1_-7_4_0_17
allSpace_63_1_-7_4_0_18
allSpace_63_1_-7_4_0_19
allSpace_63_

allSpace_63_1_-7_5_7_7
allSpace_63_1_-7_5_7_8
allSpace_63_1_-7_5_7_9
allSpace_63_1_-7_5_7_10
allSpace_63_1_-7_5_7_11
allSpace_63_1_-7_5_7_12
allSpace_63_1_-7_5_7_13
allSpace_63_1_-7_5_7_14
allSpace_63_1_-7_5_7_15
allSpace_63_1_-7_5_7_16
allSpace_63_1_-7_5_7_17
allSpace_63_1_-7_5_7_18
allSpace_63_1_-7_5_7_19
allSpace_63_1_-7_5_8_0
allSpace_63_1_-7_5_8_1
allSpace_63_1_-7_5_8_2
allSpace_63_1_-7_5_8_3
allSpace_63_1_-7_5_8_4
allSpace_63_1_-7_5_8_5
allSpace_63_1_-7_5_8_6
allSpace_63_1_-7_5_8_7
allSpace_63_1_-7_5_8_8
allSpace_63_1_-7_5_8_9
allSpace_63_1_-7_5_8_10
allSpace_63_1_-7_5_8_11
allSpace_63_1_-7_5_8_12
allSpace_63_1_-7_5_8_13
allSpace_63_1_-7_5_8_14
allSpace_63_1_-7_5_8_15
allSpace_63_1_-7_5_8_16
allSpace_63_1_-7_5_8_17
allSpace_63_1_-7_5_8_18
allSpace_63_1_-7_5_8_19
allSpace_63_1_-7_6_0_0
allSpace_63_1_-7_6_0_1
allSpace_63_1_-7_6_0_2
allSpace_63_1_-7_6_0_3
allSpace_63_1_-7_6_0_4
allSpace_63_1_-7_6_0_5
allSpace_63_1_-7_6_0_6
allSpace_63_1_-7_6_0_7
allSpace_63_1_-7_6_0_8
allSpace_63_1_

allSpace_63_1_-7_7_6_16
allSpace_63_1_-7_7_6_17
allSpace_63_1_-7_7_6_18
allSpace_63_1_-7_7_6_19
allSpace_63_1_-7_7_7_0
allSpace_63_1_-7_7_7_1
allSpace_63_1_-7_7_7_2
allSpace_63_1_-7_7_7_3
allSpace_63_1_-7_7_7_4
allSpace_63_1_-7_7_7_5
allSpace_63_1_-7_7_7_6
allSpace_63_1_-7_7_7_7
allSpace_63_1_-7_7_7_8
allSpace_63_1_-7_7_7_9
allSpace_63_1_-7_7_7_10
allSpace_63_1_-7_7_7_11
allSpace_63_1_-7_7_7_12
allSpace_63_1_-7_7_7_13
allSpace_63_1_-7_7_7_14
allSpace_63_1_-7_7_7_15
allSpace_63_1_-7_7_7_16
allSpace_63_1_-7_7_7_17
allSpace_63_1_-7_7_7_18
allSpace_63_1_-7_7_7_19
allSpace_63_1_-7_7_8_0
allSpace_63_1_-7_7_8_1
allSpace_63_1_-7_7_8_2
allSpace_63_1_-7_7_8_3
allSpace_63_1_-7_7_8_4
allSpace_63_1_-7_7_8_5
allSpace_63_1_-7_7_8_6
allSpace_63_1_-7_7_8_7
allSpace_63_1_-7_7_8_8
allSpace_63_1_-7_7_8_9
allSpace_63_1_-7_7_8_10
allSpace_63_1_-7_7_8_11
allSpace_63_1_-7_7_8_12
allSpace_63_1_-7_7_8_13
allSpace_63_1_-7_7_8_14
allSpace_63_1_-7_7_8_15
allSpace_63_1_-7_7_8_16
allSpace_63_1_-7_7_8_17
allSpace_63_

...

allSpace_63_1_-1_3_7_18
allSpace_63_1_-1_3_7_19
allSpace_63_1_-1_3_8_0
allSpace_63_1_-1_3_8_1
allSpace_63_1_-1_3_8_2
allSpace_63_1_-1_3_8_3
allSpace_63_1_-1_3_8_4
allSpace_63_1_-1_3_8_5
allSpace_63_1_-1_3_8_6
allSpace_63_1_-1_3_8_7
allSpace_63_1_-1_3_8_8
allSpace_63_1_-1_3_8_9
allSpace_63_1_-1_3_8_10
allSpace_63_1_-1_3_8_11
allSpace_63_1_-1_3_8_12
allSpace_63_1_-1_3_8_13
allSpace_63_1_-1_3_8_14
allSpace_63_1_-1_3_8_15
allSpace_63_1_-1_3_8_16
allSpace_63_1_-1_3_8_17
allSpace_63_1_-1_3_8_18
allSpace_63_1_-1_3_8_19
allSpace_63_1_-1_4_0_0
allSpace_63_1_-1_4_0_1
allSpace_63_1_-1_4_0_2
allSpace_63_1_-1_4_0_3
allSpace_63_1_-1_4_0_4
allSpace_63_1_-1_4_0_5
allSpace_63_1_-1_4_0_6
allSpace_63_1_-1_4_0_7
allSpace_63_1_-1_4_0_8
allSpace_63_1_-1_4_0_9
allSpace_63_1_-1_4_0_10
allSpace_63_1_-1_4_0_11
allSpace_63_1_-1_4_0_12
allSpace_63_1_-1_4_0_13
allSpace_63_1_-1_4_0_14
allSpace_63_1_-1_4_0_15
allSpace_63_1_-1_4_0_16
allSpace_63_1_-1_4_0_17
allSpace_63_1_-1_4_0_18
allSpace_63_1_-1_4_0_19
allSpace_63_

allSpace_63_1_-1_5_7_7
allSpace_63_1_-1_5_7_8
allSpace_63_1_-1_5_7_9
allSpace_63_1_-1_5_7_10
allSpace_63_1_-1_5_7_11
allSpace_63_1_-1_5_7_12
allSpace_63_1_-1_5_7_13
allSpace_63_1_-1_5_7_14
allSpace_63_1_-1_5_7_15
allSpace_63_1_-1_5_7_16
allSpace_63_1_-1_5_7_17
allSpace_63_1_-1_5_7_18
allSpace_63_1_-1_5_7_19
allSpace_63_1_-1_5_8_0
allSpace_63_1_-1_5_8_1
allSpace_63_1_-1_5_8_2
allSpace_63_1_-1_5_8_3
allSpace_63_1_-1_5_8_4
allSpace_63_1_-1_5_8_5
allSpace_63_1_-1_5_8_6
allSpace_63_1_-1_5_8_7
allSpace_63_1_-1_5_8_8
allSpace_63_1_-1_5_8_9
allSpace_63_1_-1_5_8_10
allSpace_63_1_-1_5_8_11
allSpace_63_1_-1_5_8_12
allSpace_63_1_-1_5_8_13
allSpace_63_1_-1_5_8_14
allSpace_63_1_-1_5_8_15
allSpace_63_1_-1_5_8_16
allSpace_63_1_-1_5_8_17
allSpace_63_1_-1_5_8_18
allSpace_63_1_-1_5_8_19
allSpace_63_1_-1_6_0_0
allSpace_63_1_-1_6_0_1
allSpace_63_1_-1_6_0_2
allSpace_63_1_-1_6_0_3
allSpace_63_1_-1_6_0_4
allSpace_63_1_-1_6_0_5
allSpace_63_1_-1_6_0_6
allSpace_63_1_-1_6_0_7
allSpace_63_1_-1_6_0_8
allSpace_63_1_

allSpace_63_1_-1_7_6_16
allSpace_63_1_-1_7_6_17
allSpace_63_1_-1_7_6_18
allSpace_63_1_-1_7_6_19
allSpace_63_1_-1_7_7_0
allSpace_63_1_-1_7_7_1
allSpace_63_1_-1_7_7_2
allSpace_63_1_-1_7_7_3
allSpace_63_1_-1_7_7_4
allSpace_63_1_-1_7_7_5
allSpace_63_1_-1_7_7_6
allSpace_63_1_-1_7_7_7
allSpace_63_1_-1_7_7_8
allSpace_63_1_-1_7_7_9
allSpace_63_1_-1_7_7_10
allSpace_63_1_-1_7_7_11
allSpace_63_1_-1_7_7_12
allSpace_63_1_-1_7_7_13
allSpace_63_1_-1_7_7_14
allSpace_63_1_-1_7_7_15
allSpace_63_1_-1_7_7_16
allSpace_63_1_-1_7_7_17
allSpace_63_1_-1_7_7_18
allSpace_63_1_-1_7_7_19
allSpace_63_1_-1_7_8_0
allSpace_63_1_-1_7_8_1
allSpace_63_1_-1_7_8_2
allSpace_63_1_-1_7_8_3
allSpace_63_1_-1_7_8_4
allSpace_63_1_-1_7_8_5
allSpace_63_1_-1_7_8_6
allSpace_63_1_-1_7_8_7
allSpace_63_1_-1_7_8_8
allSpace_63_1_-1_7_8_9
allSpace_63_1_-1_7_8_10
allSpace_63_1_-1_7_8_11
allSpace_63_1_-1_7_8_12
allSpace_63_1_-1_7_8_13
allSpace_63_1_-1_7_8_14
allSpace_63_1_-1_7_8_15
allSpace_63_1_-1_7_8_16
allSpace_63_1_-1_7_8_17
allSpace_63_

allSpace_63_1_-1_9_6_5
allSpace_63_1_-1_9_6_6
allSpace_63_1_-1_9_6_7
allSpace_63_1_-1_9_6_8
allSpace_63_1_-1_9_6_9
allSpace_63_1_-1_9_6_10
allSpace_63_1_-1_9_6_11
allSpace_63_1_-1_9_6_12
allSpace_63_1_-1_9_6_13
allSpace_63_1_-1_9_6_14
allSpace_63_1_-1_9_6_15
allSpace_63_1_-1_9_6_16
allSpace_63_1_-1_9_6_17
allSpace_63_1_-1_9_6_18
allSpace_63_1_-1_9_6_19
allSpace_63_1_-1_9_7_0
allSpace_63_1_-1_9_7_1
allSpace_63_1_-1_9_7_2
allSpace_63_1_-1_9_7_3
allSpace_63_1_-1_9_7_4
allSpace_63_1_-1_9_7_5
allSpace_63_1_-1_9_7_6
allSpace_63_1_-1_9_7_7
allSpace_63_1_-1_9_7_8
allSpace_63_1_-1_9_7_9
allSpace_63_1_-1_9_7_10
allSpace_63_1_-1_9_7_11
allSpace_63_1_-1_9_7_12
allSpace_63_1_-1_9_7_13
allSpace_63_1_-1_9_7_14
allSpace_63_1_-1_9_7_15
allSpace_63_1_-1_9_7_16
allSpace_63_1_-1_9_7_17
allSpace_63_1_-1_9_7_18
allSpace_63_1_-1_9_7_19
allSpace_63_1_-1_9_8_0
allSpace_63_1_-1_9_8_1
allSpace_63_1_-1_9_8_2
allSpace_63_1_-1_9_8_3
allSpace_63_1_-1_9_8_4
allSpace_63_1_-1_9_8_5
allSpace_63_1_-1_9_8_6
allSpace_63_1_

6415.83813095


In [49]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 67, 62, 64, 48, 70, 67, 68, 51, 70, 67, 64, 30, 70, 67, 68, 48, 967, 15760]

In [32]:
a = [1.0, 1.0, 8.0, 3.0, 'ccd_44', 'median_1', -7, 1.0, 67, 67, 62, 62, 64, 64, 48, 48, 70, 70, 67, 67, 68, 68, 51, 51, 70, 70, 67, 67, 64, 64, 30, 30, 70, 70, 67, 67, 68, 68, 48, 48, 967, 15760]

In [33]:
a[24]

70

In [14]:
tup = [(1,2,3),(4,5,6)]
rdd = sc.parallelize(tup)
kepler = rdd.map(lambda x: Row(one=int(x[0]),two=int(x[1]),three=int(x[2])))
schemaKepler = sqlContext.createDataFrame(kepler)
schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='testTab',user='mj1e16',password='[sqlT1G3R]').mode('append').save()


In [7]:
a = sqlContext.sql("show tables").show()
print(a)

+--------+----------+-----------+
|database| tableName|isTemporary|
+--------+----------+-----------+
|        |starlist_1|       true|
|        |starlist_2|       true|
|        |starlist_3|       true|
|        |starlist_4|       true|
|        |starlist_5|       true|
|        |starlist_6|       true|
|        |starlist_7|       true|
+--------+----------+-----------+

None


In [15]:
def test():
    row = [('tabName',1,1),('tabName2',1,2)]
    rdd = sc.parallelize(row)
    kepler = rdd.map(lambda x: Row(tabName=x[0], xyRegion=int(x[1]),magRegion=int(x[2])))
    schemaKepler = sqlContext.createDataFrame(kepler)
    df = schemaKepler.toPandas()
    print(df)

In [25]:
def testPartial(value):
    return (value) #,value)

In [26]:
p = ThreadPool(6)
values = [0,1,2,3,4,5]
answer = []
for x in range(5):
    answer.extend(p.map(testPartial,values))
print(answer)

[0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]


In [26]:
import sys

In [27]:
sys.getsizeof(answer)

336

In [12]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
    maxVal = df2['MAG_BEST'].max()
    minVal = df2['MAG_BEST'].min()
#     df4 = df2.sort_values('MAG_BEST')
#     return df4
    print(minVal,maxVal)

In [32]:
def findMagRange(catName):
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    #df3 = df[df['MAG_BEST'] == 99.0000]
#     maxVal = df2['MAG_BEST'].max()
#     minVal = df2['MAG_BEST'].min()
    df4 = df2.sort_values('MAG_BEST')
    return df4
#     print(minVal,maxVal)

In [15]:
for x in range(6):
    print('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))
    findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

/home/mj1e16/sextractor/sextractor-master/config/test0.cat
/home/mj1e16/sextractor/sextractor-master/config/test1.cat
/home/mj1e16/sextractor/sextractor-master/config/test2.cat
/home/mj1e16/sextractor/sextractor-master/config/test3.cat
/home/mj1e16/sextractor/sextractor-master/config/test4.cat
/home/mj1e16/sextractor/sextractor-master/config/test5.cat


In [33]:
x = 2
df = findMagRange('/home/mj1e16/sextractor/sextractor-master/config/test{}.cat'.format(x))

In [35]:
df

,NUMBER,MAG_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY
3025,3026,13.0256,782.0090,185.0009,0.352
2869,2870,13.0305,1052.8906,184.9989,0.036
13573,13574,13.0327,1052.7808,946.0535,0.125
13344,13345,13.0328,239.7816,946.0737,0.031
6969,6970,13.0329,239.5148,437.8912,0.103
11193,11194,13.0329,239.6400,691.9606,0.054
13666,13667,13.0333,781.5560,946.0166,0.260
6968,6969,13.0335,781.4474,438.1546,0.157
7010,7011,13.0338,510.7693,437.9970,0.031
2859,2860,13.0340,239.8327,185.1199,0.095


In [49]:
magRange = np.linspace(15,30,16)

In [50]:
magRange

array([ 15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,
        26.,  27.,  28.,  29.,  30.])

In [57]:
catName = 'test0.cat'

assoc = Table.read(catName,format='ascii.sextractor')
t0 = time.time()
df = assoc.to_pandas()
#df2 = df[df['MAG_BEST'] != 99.0000]
print(time.time()-t0) #, totGross)


0.00117707252502


In [56]:
t0 = time.time()
df_spark = sqlContext.createDataFrame(df)
print(time.time()-t0) #, totGross)

t0 = time.time()
totGross = df_spark.count()
print(time.time()-t0, totGross)

1.73592019081
(0.06544804573059082, 16094)


In [59]:
!head test0.cat

#   1 NUMBER                 Running object number                                     
#   2 MAG_BEST               Best of MAG_AUTO and MAG_ISOCOR                            [mag]
#   3 X_IMAGE                Object position along x                                    [pixel]
#   4 Y_IMAGE                Object position along y                                    [pixel]
#   5 ELLIPTICITY            1 - B_IMAGE/A_IMAGE                                       
         1  20.7173    744.5268     19.6496    0.438
         2  99.0000   1000.7009     19.5017    0.285
         3  99.0000    133.3164     20.2157    0.325
         4  21.8619    138.0000     20.4564    0.499
         5  21.5303    779.9139     23.0000    0.633


In [27]:
x = 0
vals = [[x],[1],[2]]
val = [[x],[3]]
print(val)
x = 1
val = [[x],[3]]
print(val)
vals.append(val[0])

[[0], [3]]
[[1], [3]]


In [4]:
import numpy as np 
smallName = 'test'

In [5]:
valListminus1 = [1,3,5,7,9,11]
for vals in (valListminus1):
    valList = [np.linspace(1,10,10),np.linspace(1,9,9),np.linspace(8,128,16),[vals]]

    finalTableTuples = []
    #p = ThreadPool(1)
    #t0 = time.time()
    for x0 in range(3): #len(valList[0])):
        for x1 in range(3): #len(valList[1])):
                for x2 in range(3): #len(valList[2])):
                    name = smallName+'_'+str(x0)+'_'+str(x1)+'_'+str(x2)+'_'
                    fullname = [name]*len(valList[3])
                    fullValList = []
                    for x3 in range(len(valList[3])):
                        fullname[x3] += str(x3)
                        print(fullname[x3])
                        fullValList.append([valList[0][x0],valList[1][x1],valList[2][x2],valList[3][x3]])

                        #alltogethernow(fullValList[x4],fullname,imageName,provDir)
                    print(fullValList)
#                     try:
#                         finalTableTuples.extend(p.map(partial(alltogethernow,tableName=fullname,IMAGE=simImage,provDir=provDir),fullValList))

test_0_0_0_0
[[1.0, 1.0, 8.0, 1]]
test_0_0_1_0
[[1.0, 1.0, 16.0, 1]]
test_0_0_2_0
[[1.0, 1.0, 24.0, 1]]
test_0_1_0_0
[[1.0, 2.0, 8.0, 1]]
test_0_1_1_0
[[1.0, 2.0, 16.0, 1]]
test_0_1_2_0
[[1.0, 2.0, 24.0, 1]]
test_0_2_0_0
[[1.0, 3.0, 8.0, 1]]
test_0_2_1_0
[[1.0, 3.0, 16.0, 1]]
test_0_2_2_0
[[1.0, 3.0, 24.0, 1]]
test_1_0_0_0
[[2.0, 1.0, 8.0, 1]]
test_1_0_1_0
[[2.0, 1.0, 16.0, 1]]
test_1_0_2_0
[[2.0, 1.0, 24.0, 1]]
test_1_1_0_0
[[2.0, 2.0, 8.0, 1]]
test_1_1_1_0
[[2.0, 2.0, 16.0, 1]]
test_1_1_2_0
[[2.0, 2.0, 24.0, 1]]
test_1_2_0_0
[[2.0, 3.0, 8.0, 1]]
test_1_2_1_0
[[2.0, 3.0, 16.0, 1]]
test_1_2_2_0
[[2.0, 3.0, 24.0, 1]]
test_2_0_0_0
[[3.0, 1.0, 8.0, 1]]
test_2_0_1_0
[[3.0, 1.0, 16.0, 1]]
test_2_0_2_0
[[3.0, 1.0, 24.0, 1]]
test_2_1_0_0
[[3.0, 2.0, 8.0, 1]]
test_2_1_1_0
[[3.0, 2.0, 16.0, 1]]
test_2_1_2_0
[[3.0, 2.0, 24.0, 1]]
test_2_2_0_0
[[3.0, 3.0, 8.0, 1]]
test_2_2_1_0
[[3.0, 3.0, 16.0, 1]]
test_2_2_2_0
[[3.0, 3.0, 24.0, 1]]
test_0_0_0_0
[[1.0, 1.0, 8.0, 3]]
test_0_0_1_0
[[1.0, 1.0, 16.0,

In [18]:
np.linspace(-7,-1,7)

array([-7., -6., -5., -4., -3., -2., -1.])

In [82]:
a = [1,7,17,177]
a.index(7)

1